# Example 2: Parameter Optimization

This example demonstrates how to use parameter sweeps to find optimal processing parameters.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from frame_overlap import Workflow

%matplotlib inline

## Optimize Noise Power

Find the optimal `noise_power` parameter for Wiener filtering.

In [ ]:
# Set up parameter sweep
results = (Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .groupby('noise_power', low=0.05, high=0.5, num=15)
    .reconstruct(kind='wiener')
    .analyze(xs='iron', vary_background=True, vary_response=True)
    .run())

# Remove failed runs
results_clean = results.dropna(subset=['redchi2'])
print(f"Successful runs: {len(results_clean)}/{len(results)}")
print("\nResults:")
print(results_clean[['noise_power', 'chi2', 'redchi2', 'aic']].head())

## Find Optimal Parameters

In [ ]:
# Find best by reduced chi-squared
best_chi2 = results_clean.loc[results_clean['redchi2'].idxmin()]
print("Best by χ²/dof:")
print(f"  noise_power: {best_chi2['noise_power']:.4f}")
print(f"  χ²/dof: {best_chi2['redchi2']:.2f}")

# Find best by AIC (Akaike Information Criterion)
best_aic = results_clean.loc[results_clean['aic'].idxmin()]
print("\nBest by AIC:")
print(f"  noise_power: {best_aic['noise_power']:.4f}")
print(f"  AIC: {best_aic['aic']:.1f}")

## Visualize Parameter Space

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Reduced chi-squared
ax = axes[0, 0]
ax.plot(results_clean['noise_power'], results_clean['redchi2'], 'o-', markersize=6)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='χ²/dof = 1')
ax.axvline(x=best_chi2['noise_power'], color='red', linestyle=':', alpha=0.7, label='Optimal')
ax.set_xlabel('Noise Power')
ax.set_ylabel('Reduced χ²')
ax.set_title('Fit Quality vs Noise Power')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: AIC
ax = axes[0, 1]
ax.plot(results_clean['noise_power'], results_clean['aic'], 's-', color='orange', markersize=6)
ax.axvline(x=best_aic['noise_power'], color='red', linestyle=':', alpha=0.7, label='Optimal')
ax.set_xlabel('Noise Power')
ax.set_ylabel('AIC')
ax.set_title('AIC vs Noise Power (lower is better)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: BIC
ax = axes[1, 0]
ax.plot(results_clean['noise_power'], results_clean['bic'], '^-', color='green', markersize=6)
ax.set_xlabel('Noise Power')
ax.set_ylabel('BIC')
ax.set_title('BIC vs Noise Power (lower is better)')
ax.grid(True, alpha=0.3)

# Plot 4: Chi-squared (not reduced)
ax = axes[1, 1]
ax.semilogy(results_clean['noise_power'], results_clean['chi2'], 'd-', color='purple', markersize=6)
ax.set_xlabel('Noise Power')
ax.set_ylabel('χ² (log scale)')
ax.set_title('Chi-Squared vs Noise Power')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Fitted Parameters vs Noise Power

Examine how fitted parameters change with noise_power.

In [ ]:
# Get parameter columns
param_cols = [col for col in results_clean.columns if col.startswith('param_') and not col.endswith('_err')]

# Plot first 4 parameters
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for i, param_col in enumerate(param_cols[:4]):
    param_name = param_col.replace('param_', '')
    err_col = f"{param_col}_err"
    
    if err_col in results_clean.columns:
        axes[i].errorbar(results_clean['noise_power'], 
                        results_clean[param_col],
                        yerr=results_clean[err_col],
                        fmt='o-', capsize=3, capthick=1)
    else:
        axes[i].plot(results_clean['noise_power'], results_clean[param_col], 'o-')
    
    axes[i].axvline(x=best_chi2['noise_power'], color='red', linestyle=':', alpha=0.5)
    axes[i].set_xlabel('Noise Power')
    axes[i].set_ylabel(param_name)
    axes[i].set_title(f'{param_name} vs Noise Power')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Run with Optimal Parameters

In [ ]:
# Re-run with optimal noise_power
optimal_noise = best_chi2['noise_power']

wf_optimal = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)
result = (wf_optimal
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .reconstruct(kind='wiener', noise_power=optimal_noise)
    .analyze(xs='iron', vary_background=True, vary_response=True))

print(f"\nOptimized workflow complete!")
print(f"Noise power: {optimal_noise:.4f}")
print(f"Final χ²/dof: {wf_optimal.result.redchi:.2f}")

# Plot optimized result
wf_optimal.plot()
plt.suptitle(f'Optimized Result (noise_power={optimal_noise:.4f})', y=1.02)
plt.show()

## Key Takeaways

- **`.groupby()`**: Easy parameter space exploration
- **Progress tracking**: Automatic tqdm progress bars
- **Rich results**: DataFrame with all metrics (χ², AIC, BIC, parameters)
- **Error handling**: Continues even if individual runs fail
- **Multiple criteria**: Compare χ²/dof, AIC, and BIC for model selection
- **Parameter stability**: Examine how fitted values change with sweep parameter